In [9]:
# https://www.geeksforgeeks.org/linear-search/?ref=outind

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Binary Search Algorithm

In [10]:
def binary_search(l, value): # logN
  low = 0
  high = len(l)-1
  while low <= high: 
    mid = (low+high)//2
    if l[mid] > value: high = mid-1
    elif l[mid] < value: low = mid+1
    else: return mid
  return -1

## Frequency Count Method

In [11]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(binary_search)

  # create a list with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  arr = sorted(input)
  lp_wrapper(arr, arr[-1]+1)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [12]:
resultados_com_menor_loss = []
repeat = 5
registros = []

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=["log", "square", "cube"],
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=["log", "square", "cube"],
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=["log", "square", "cube"],
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-8, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           1.096e+02  1.594e+01  y = x₀                                        3           1.066e+02  1.396e-02  y = (-0.82342 + x₀)                           5           1.046e+02  9.507e-03  y = (log(log(1.1258)) + x₀)                   6           2.059e+01  1.625e+00  y = log(cube(cube(x₀) + x₀))                  ---------------------------------------------------------------------------------------------------
0.3

In [13]:
registros_ = registros

In [14]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [15]:
def salvar_dados(dados, key, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)
  else:
    dados_exist = {}
    
  valor_original = dados_exist.get(key)
  if valor_original == None:
    dados_exist.update({key: [dados]})
  else:
    if isinstance(valor_original, list):
      valor_original.append(dados)
    else:
      dados_exist.update({key: [dados]})

  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)

caminho_arquivo = 'dados.pickle'
novos_dados = lista_melhor_valor[0:3] + [lista_melhor_valor[4]]

salvar_dados(novos_dados, 'binary_search_logx', caminho_arquivo)

In [24]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = 'dados.pickle'
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items():
  print('\u25CF', k)
  for index, item in enumerate(v):
    if index == len(v)-1:
       print('└─', item)
    else:
      print('├─', item)
  print('==========================')

Conteúdo do arquivo pickle:
● binary_search_logx
├─ [6, 1.2914093, 1.37589542429269, log(x0**6) + 5.574131]
└─ [5, 1.5291592, 2.19479656667366, log(x0)**2 + 13.966276]
